Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import psycopg2

In [2]:
engagement_df = pd.read_csv('engagement_data.csv')
engagement_df.head()

,Customer ID,Handset Type,Total Duration (ms),Total Download (Bytes),Total Upload (Bytes),Session Frequency,Total Traffic (Bytes),Engagement Cluster
0,3.360100e+10,Huawei P20 Lite Huawei Nova 3E,116720.0,455.940801,36.053108,1,491.993908,1
1,3.360100e+10,Apple iPhone 7 (A1778),181230.0,120.755184,36.104459,1,156.859643,1
2,3.360100e+10,undefined,134969.0,556.659663,39.306820,1,595.966483,1
3,3.360101e+10,Apple iPhone 5S (A1457),49878.0,401.993172,20.327526,1,422.320698,1
4,3.360101e+10,Apple iPhone Se (A1723),37104.0,1363.130417,94.280527,2,1457.410944,0


In [7]:
experience_df = pd.read_csv('experience_data.csv')
experience_df.head()

,MSISDN/Number,Handset Type,Avg TCP Retransmission (Bytes),Avg RTT (ms),Avg Throughput (kbps),Experience Cluster
0,3.360100e+10,Huawei P20 Lite Huawei Nova 3E,21.777861,46.000000,76.0,0
1,3.360100e+10,Apple iPhone 7 (A1778),21.777861,31.000000,99.0,0
2,3.360100e+10,undefined,21.777861,123.828174,97.0,0
3,3.360101e+10,Apple iPhone 5S (A1457),0.772118,84.000000,248.0,1
4,3.360101e+10,Apple iPhone Se (A1723),15.574346,59.500000,20248.5,0


Task 4.1a - Calculate Engagement Score

In [3]:
# Group by the cluster label and calculate the mean of relevant features to get cluster centers
engagement_clusters_df = engagement_df.groupby('Engagement Cluster')[['Session Frequency', 'Total Duration (ms)', 'Total Traffic (Bytes)']].mean()

# Convert the DataFrame to a numpy array if you need it in the same format as cluster centers
engagement_clusters = engagement_clusters_df.values


In [4]:
from sklearn.metrics.pairwise import euclidean_distances

def calculate_engagement_score(row):
    return euclidean_distances([row], [engagement_clusters[0]])[0][0]

engagement_df['engagement_score'] = engagement_df.apply(lambda row: calculate_engagement_score(row[['Session Frequency', 'Total Duration (ms)', 'Total Traffic (Bytes)']]), axis=1)


In [5]:
engagement_df.rename(columns={'Customer ID': 'MSISDN/Number'}, inplace=True)
engagement_df.head(10)

,MSISDN/Number,Handset Type,Total Duration (ms),Total Download (Bytes),Total Upload (Bytes),Session Frequency,Total Traffic (Bytes),Engagement Cluster,engagement_score
0,3.360100e+10,Huawei P20 Lite Huawei Nova 3E,116720.0,455.940801,36.053108,1,491.993908,1,97542.728966
1,3.360100e+10,Apple iPhone 7 (A1778),181230.0,120.755184,36.104459,1,156.859643,1,33043.507886
2,3.360100e+10,undefined,134969.0,556.659663,39.306820,1,595.966483,1,79293.433097
3,3.360101e+10,Apple iPhone 5S (A1457),49878.0,401.993172,20.327526,1,422.320698,1,164384.304020
4,3.360101e+10,Apple iPhone Se (A1723),37104.0,1363.130417,94.280527,2,1457.410944,0,177157.487880
5,3.360101e+10,Samsung Galaxy A8 (2018),253983.0,555.207972,60.009249,2,615.217221,0,39724.484866
6,3.360101e+10,Huawei Mate 10 Pro Porsche Design Huawei Mate 10,128360.0,553.041208,68.482286,2,621.523493,0,85902.191305
7,3.360101e+10,Samsung Galaxy S8 Plus (Sm-G955F),86399.0,294.085078,38.575279,1,332.660357,1,127864.143295
8,3.360101e+10,undefined,495702.0,896.560173,93.572016,2,990.132189,0,281440.958461
9,3.360102e+10,Apple iPhone X (A1865),124854.0,685.372753,47.091008,1,732.463761,1,89407.666287


Cell 4: Task 4.1b - Calculate Experience Score


In [9]:
# Group by the cluster label and calculate the mean of relevant features to get cluster centers
experience_clusters_df = experience_df.groupby('Experience Cluster')[['Avg TCP Retransmission (Bytes)', 'Avg RTT (ms)','Avg Throughput (kbps)']].mean()

# Convert the DataFrame to a numpy array if you need it in the same format as cluster centers
experience_clusters = experience_clusters_df.values

In [10]:

def calculate_experience_score(row):
    return euclidean_distances([row], [experience_clusters[0]])[0][0]

# Apply this function to your DataFrame to calculate the engagement score
experience_df['experience_score'] = experience_df.apply(lambda row: calculate_experience_score(row[['Avg TCP Retransmission (Bytes)', 'Avg RTT (ms)','Avg Throughput (kbps)']]), axis=1)


In [11]:
experience_df.head(10)

,MSISDN/Number,Handset Type,Avg TCP Retransmission (Bytes),Avg RTT (ms),Avg Throughput (kbps),Experience Cluster,experience_score
0,3.360100e+10,Huawei P20 Lite Huawei Nova 3E,21.777861,46.000000,76.0,0,851.855914
1,3.360100e+10,Apple iPhone 7 (A1778),21.777861,31.000000,99.0,0,829.640778
2,3.360100e+10,undefined,21.777861,123.828174,97.0,0,831.259229
3,3.360101e+10,Apple iPhone 5S (A1457),0.772118,84.000000,248.0,1,679.466421
4,3.360101e+10,Apple iPhone Se (A1723),15.574346,59.500000,20248.5,0,19321.367878
5,3.360101e+10,Samsung Galaxy A8 (2018),11.275836,73.914087,3954.0,1,3026.880116
6,3.360101e+10,Huawei Mate 10 Pro Porsche Design Huawei Mate 10,10.944046,26.500000,21256.5,1,20329.430471
7,3.360101e+10,Samsung Galaxy S8 Plus (Sm-G955F),0.771330,52.000000,1247.0,1,321.812985
8,3.360101e+10,undefined,21.777861,123.828174,94.5,0,833.755888
9,3.360102e+10,Apple iPhone X (A1865),21.008103,62.000000,146.0,0,781.371047


Task 4.2 - Calculate Satisfaction Score and Report Top 10 Satisfied Customers

Merge the DataFrames

In [22]:
# Merge the two DataFrames on the 'MSISDN/Number' column
df_merged = pd.merge(engagement_df[['MSISDN/Number','Handset Type','engagement_score']],
                     experience_df[['MSISDN/Number','experience_score']],
                     on='MSISDN/Number',
                     how='inner')


Calculate the Satisfaction Score

In [23]:
# Calculate satisfaction score as the average of engagement and experience scores
df_merged['satisfaction_score'] = df_merged[['engagement_score', 'experience_score']].mean(axis=1)


Identify the Top 10 Satisfied Customers

In [24]:
# Top 10 satisfied customers
top_10_satisfied = df_merged.nlargest(10, 'satisfaction_score')

top_10_satisfied


,MSISDN/Number,Handset Type,engagement_score,experience_score,satisfaction_score
13123,3.362578e+10,Samsung Galaxy A8 (2018),1.833949e+07,7501.717230,9.173498e+06
6404,3.361489e+10,Samsung Galaxy J5 (Sm-J530),9.752639e+06,15911.096399,4.884275e+06
92584,3.376054e+10,Samsung Galaxy J5 (Sm-J530),9.065176e+06,4794.387283,4.534985e+06
13467,3.362632e+10,Samsung Galaxy J3 (Sm-J330),8.577668e+06,5814.257405,4.291741e+06
64883,3.366716e+10,Samsung Galaxy J5 (Sm-J530),8.530654e+06,8257.293905,4.269456e+06
50096,3.366284e+10,Tp-Link Technolo. Tl-Mr6400,6.400010e+06,24425.440005,3.212218e+06
658,3.360313e+10,Samsung Galaxy S9 Plus (Sm-G965F),6.073501e+06,11305.921109,3.042404e+06
56951,3.366469e+10,undefined,6.074470e+06,845.906573,3.037658e+06
65787,3.366746e+10,Apple iPhone 8 Plus (A1897),5.435621e+06,847.627299,2.718234e+06
92239,3.376041e+10,undefined,5.107408e+06,18270.860658,2.562840e+06


Task 4.3 - Build a Regression Model to Predict Satisfaction Score

In [27]:
# Define features and target
features = df_merged[['engagement_score', 'experience_score']]
target = df_merged['satisfaction_score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Build and train the regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
print(f"Model R^2: {model.score(X_test, y_test)}")


Model R^2: 1.0


Cell 7: Task 4.4 - K-Means Clustering on Engagement & Experience Score (k=2)

In [28]:
# Perform k-means clustering (k=2) on engagement and experience scores
kmeans_satisfaction = KMeans(n_clusters=2, random_state=42)
df_merged['satisfaction_cluster'] = kmeans_satisfaction.fit_predict(df_merged[['engagement_score', 'experience_score']])

# Visualize or output cluster assignments
df_merged[['MSISDN/Number', 'satisfaction_cluster']].head(10)


,MSISDN/Number,satisfaction_cluster
0,3.360100e+10,0
1,3.360100e+10,0
2,3.360100e+10,0
3,3.360101e+10,0
4,3.360101e+10,0
5,3.360101e+10,0
6,3.360101e+10,0
7,3.360101e+10,0
8,3.360101e+10,0
9,3.360102e+10,0


Task 4.5 - Aggregate Average Satisfaction & Experience Score per Cluster

In [29]:
# Aggregate average satisfaction and experience score per cluster
cluster_aggregation = df_merged.groupby('satisfaction_cluster').agg({
    'satisfaction_score': 'mean',
    'experience_score': 'mean',
    'MSISDN/Number': 'count'
}).rename(columns = {'MSISDN/Number': 'Number of Users'}).reset_index()

print("Cluster Summary:")
print(cluster_aggregation)

Cluster Summary:
   satisfaction_cluster  satisfaction_score  experience_score  Number of Users
0                     0        6.408947e+04       8589.944186           106167
1                     1        1.019538e+06       9797.665423              304


In [31]:
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

In [32]:

# Establish a connection to the PostgreSQL database
        # Create a connection string
connection_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

        # Create an SQLAlchemy engine
engine = create_engine(connection_string)

# Export the dataframe to a table in PostgreSQL
df_merged[['MSISDN/Number','Handset Type','engagement_score', 'experience_score', 'satisfaction_score']].to_sql(
    'customer_satisfaction', 
    con=engine, 
    if_exists='replace', 
    index=False
)

# Run a SELECT query to verify
result = pd.read_sql("SELECT * FROM customer_satisfaction LIMIT 10", con=engine)
result.head()


,MSISDN/Number,Handset Type,engagement_score,experience_score,satisfaction_score
0,3.360100e+10,Huawei P20 Lite Huawei Nova 3E,97542.728966,851.855914,49197.292440
1,3.360100e+10,Apple iPhone 7 (A1778),33043.507886,829.640778,16936.574332
2,3.360100e+10,undefined,79293.433097,831.259229,40062.346163
3,3.360101e+10,Apple iPhone 5S (A1457),164384.304020,679.466421,82531.885220
4,3.360101e+10,Apple iPhone Se (A1723),177157.487880,19321.367878,98239.427879
